In [ ]:
### save NSD sythetic stimuli as jpg

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import h5py
from PIL import Image
import numpy as np

In [4]:
nsd_root = os.path.join(os.getenv('DATASETS_ROOT'), 'NaturalScenesDataset', 'derivatives','stimuli_metadata')
save_root = os.path.join(nsd_root, 'nsdsynthetic_jpg')
if not os.path.exists(save_root):
    os.makedirs(save_root)
save_root_cropped = os.path.join(nsd_root, 'nsdsynthetic_cropped_jpg')
if not os.path.exists(save_root_cropped):
    os.makedirs(save_root_cropped)
numsubjects=8
start = (1360 - 714) // 2  #where to start the crop of a (714, 1360, 3) image (width, height, color)

In [5]:
#integers 1-220 in the design tsv files correspond to nsdsynthetic_stimuli.hdf5 block and 221-284 corresond to 'nsdsynthetic_colorstimuli_subj0X.hdf5' block
count = 0
with h5py.File(os.path.join(nsd_root,'nsdsynthetic_stimuli.hdf5'), 'r') as file:
    numimages = file['imgBrick'].shape[0]
    for _ in range(numimages):
        stim_name = f"{count+1:03}.jpg"
        img = file['imgBrick'][count,:,:,:]
        #save image
        Image.fromarray(img.astype(np.uint8)).save(os.path.join(save_root, stim_name))
        
        img_cropped = img[:, start:start+714, :]  #crop to (714, 714, 3)
        Image.fromarray(img_cropped.astype(np.uint8)).save(os.path.join(save_root_cropped, stim_name))
        count += 1

In [6]:
for idx in range(64):
    #average each image over the subjects
    img = np.zeros((714, 1360, 3))
    stim_name = f"{count+1:03}.jpg"
    for subject in range(1,numsubjects+1):
        with h5py.File(os.path.join(nsd_root,f'nsdsynthetic_colorstimuli_subj{subject:02}.hdf5'), 'r') as file:
            img += file['imgBrick'][idx,:,:,:]
    img = img/numsubjects #average
    Image.fromarray(img.astype(np.uint8)).save(os.path.join(save_root, stim_name))
    img_cropped = img[:, start:start+714, :]  #crop to (714, 714, 3)
    Image.fromarray(img_cropped.astype(np.uint8)).save(os.path.join(save_root_cropped, stim_name))
    count+=1